In [ ]:
#API KEY 974b9b83d36dc48184eb17242d335148
import requests
import pandas as pd

base_url = 'https://financialmodelingprep.com/api/v3'
data_type = 'ratios'
ticker = 'COST'
#My API Key from ACCT I created
API_KEY = '974b9b83d36dc48184eb17242d335148'
url = f'{base_url}/{data_type}/{ticker}?limit=120&apikey={API_KEY}'
r = requests.get(url)

In [ ]:
r #this checks the response from the server. 429 - too many requests, 200 - works, 404- not found

In [ ]:
#Use JSON to return data that is consumable to user
import json
json.loads(r.text)

In [ ]:
#ratios = ['debtToEquity', 'currentRatio', 'quickRatio', 'returnOnEquity', 'grossProfitMargin']

info = json.loads(r.text)
info[1]['debtEquityRatio']

In [ ]:
#5 most relevant ratios
ratio_data = {info[1]['debtEquityRatio'], info[1]['returnOnEquity'],  info[1]['netProfitMargin'], info[1]['quickRatio'], info[1]['dividendYield'],}

In [ ]:
#ratio df
ratio_df = pd.DataFrame(ratio_data).T

In [ ]:
#ratios in Df listed out
ratio_df

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Define API parameters
base_url = 'https://financialmodelingprep.com/api/v3'
data_type = 'historical-price-full'
ticker = 'COST'
API_KEY = '974b9b83d36dc48184eb17242d335148'
url = f'{base_url}/{data_type}/{ticker}?apikey={API_KEY}'

# Fetch data from the API
response = requests.get(url)
data = response.json()

# Convert data to DataFrame
price_df = pd.DataFrame(data['historical'])

# Convert date column to datetime format
price_df['date'] = pd.to_datetime(price_df['date'])

# Plot stock prices
plt.figure(figsize=(10, 6))
plt.plot(price_df['date'], price_df['close'], label='Closing Price', color='blue')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title(f'Stock Price for {ticker}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Define API parameters
base_url = 'https://financialmodelingprep.com/api/v3'
data_type = 'cash-flow-statement'
ticker = 'COST'
API_KEY = '974b9b83d36dc48184eb17242d335148'
url = f'{base_url}/{data_type}/{ticker}?apikey={API_KEY}'

# Fetch data from the API
response = requests.get(url)
data = response.json()

# Convert data to DataFrame
cash_flow_df = pd.DataFrame(data)

# Selecting relevant columns for visualization
cash_flow_df = cash_flow_df[['date', 'freeCashFlow']]

# Convert date column to datetime format
cash_flow_df['date'] = pd.to_datetime(cash_flow_df['date'])

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(cash_flow_df['date'], cash_flow_df['freeCashFlow'], label='Free Cash Flow', color='green')
plt.xlabel('Date')
plt.ylabel('Free Cash Flow ($)')
plt.title(f'Free Cash Flow for {ticker}')
plt.legend()
plt.grid(True)
plt.show()

In [8]:
from PIL import Image
#https://github.com/hpop13/HunterDCF-/blob/0d5172472857b11aad68be7a151d766f07a81be8/FINANCEDCFEXAM.png
#c:\Users\hunte\Downloads\FINANCEDCFEXAM.png

# Read the image
im1 = Image.open("https://github.com/hpop13/HunterDCF-/blob/0d5172472857b11aad68be7a151d766f07a81be8/FINANCEDCFEXAM.png")

im1.show()

OSError: [Errno 22] Invalid argument: 'https://github.com/hpop13/HunterDCF-/blob/0d5172472857b11aad68be7a151d766f07a81be8/FINANCEDCFEXAM.png'

In [ ]:
import requests

# Function to calculate the DCF of an investment
#inital_cash_flow is pulled from API, so is growth_rate
#discount_rate, years, terminal_growth_rate are manual inputs
def calculate_dcf(initial_cash_flow, growth_rate, discount_rate, years, terminal_growth_rate):
    #initialize the variable "dcf"@ 0; stores cumulative DCFs
    dcf = 0
     #based on year input 
    for year in range(1, years + 1):
        #project out using our  growth rate and then discount using r
        #using 1 CF
        discounted_cash_flow = initial_cash_flow * (1 + growth_rate) ** year / ((1 + discount_rate) ** year)
        dcf += discounted_cash_flow
        #derived using perpetuity formula in finance
        #represents the present value of an infinite series of future cash flows.
    terminal_value = initial_cash_flow * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    # This line adds the terminal value to the cumulative discounted cash flow (dcf). This accounts for the cash flows beyond the projection period.
    dcf += terminal_value
    return dcf

# Function to fetch financial data from the API
def get_financial_data(ticker):
    API_KEY = '974b9b83d36dc48184eb17242d335148'
    base_url = 'https://financialmodelingprep.com/api/v3'
    income_url = f'{base_url}/financials/income-statement/{ticker}?apikey={API_KEY}'
    cash_flow_url = f'{base_url}/financials/cash-flow-statement/{ticker}?apikey={API_KEY}'
    BS_url = f'{base_url}/financials/balance-sheet-statement/{ticker}?apikey={API_KEY}'
    
    try:
        #request sent by web browser to the requests library then to a server to retrieve financial data
        #this line sends an HTTP GET request to the URL specified in income_url
        income_response = requests.get(income_url)
        cash_flow_response = requests.get(cash_flow_url)
        bs_response = requests.get(BS_url)

        #parses the JSON response received from the server into a Python dictionary.
        #attempts to retrieve the value associated with the key 'financials' from the parsed JSON data
        #When you receive data in the form of JSON, parsing involves converting that JSON string into a data structure that your programming language can manipulate. 
        income_data = income_response.json().get('financials', [])
        cash_flow_data = cash_flow_response.json().get('financials', [])
        bs_data = bs_response.json().get('financials', [])

        #his condition checks if all the necessary financial data (income_data, cash_flow_data, and bs_data) have been successfully retrieved from the API
        if income_data and cash_flow_data and bs_data:
            #This line extracts the value of "Free Cash Flow" from the first item of the cash_flow_data list (assuming it's present) and converts it to a floating-point number. If "Free Cash Flow" is not found, it defaults to 0.
            initial_cash_flow = float(cash_flow_data[0].get('Free Cash Flow', 0))
            #This line calculates the growth rate based on revenue data from the income_data list. It computes the percentage change in revenue from one period to the next.
            growth_rate = (float(income_data[0].get('Revenue', 0)) - float(income_data[1].get('Revenue', 0))) / float(income_data[1].get('Revenue', 1))
            #extracts the value of "Net Debt"
            net_debt = float(bs_data[0].get('Net Debt', 0))
            shares_outstanding = float(input("Enter the number of shares issued: "))  # Prompt user input for shares issued
            return initial_cash_flow, growth_rate, net_debt, shares_outstanding
        else:
            #error message indicating the nature of the problem and then returns None values for all variables, indicating that there was a failure in fetching the data.
            return None, None, None, None
    except (KeyError, ValueError):
        return None, None, None, None
    except requests.exceptions.RequestException as e:
        print("Error fetching financial data:", e)
        return None, None, None, None

# Get user input for assumptions
ticker = input("Enter the ticker symbol: ")
discount_rate = float(input("Enter the discount rate (as a decimal): "))
years = int(input("Enter the number of years for the DCF analysis: "))
terminal_growth_rate = float(input("Enter the terminal growth rate (as a decimal): "))

# Get financial data and assumptions
initial_cash_flow, growth_rate, net_debt, shares_outstanding = get_financial_data(ticker)

if initial_cash_flow is not None and growth_rate is not None:
    # Calculate DCF
    #DCF Built in function
    dcf_value = calculate_dcf(initial_cash_flow, growth_rate, discount_rate, years, terminal_growth_rate)
    print(f"The discounted cash flow (DCF) after {years} years is: {dcf_value:.2f}")

    if shares_outstanding is not None and shares_outstanding != 0:
        # Calculate equity value per share
        equity_value_per_share = dcf_value / shares_outstanding
        print(f"The equity value per share is: {equity_value_per_share:.2f}")
    else:
        print("Failed to retrieve shares outstanding data or shares outstanding is 0.")
else:
    print("Failed to retrieve financial data for the given ticker symbol.")
